In [1]:
import torchvision
import torch

# model = torchvision.models.resnet18(pretrained=False)
# model.fc = torch.nn.Linear(512, 1)

model = torchvision.models.squeezenet1_1(pretrained=False)
model.classifier[1] = torch.nn.Conv2d(512, 1, kernel_size=1)
model.num_classes = 1

model.load_state_dict(torch.load('roadFollowing_V3_squeeze_conv.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

In [1]:
# Optimized Tensor RT net
import torch
device = torch.device('cuda')
import torch
from torch2trt import TRTModule

model = TRTModule()
model.load_state_dict(torch.load('squeezenet_V3_GPU_tensorRT.pth'))
model = model.eval().half()

In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [3]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera(fps=12)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [4]:
from jetbot import Robot
robot = Robot()

In [5]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
brake_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='braking')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, brake_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='braking', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [6]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [7]:
angle = 0.0
angle_last = 0.0
angle_history = []
import time

def execute(change):
    global angle, angle_last
    image = change['new']
    
    slope = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    
    angle = np.deg2rad(slope - 90)
    
    angle_history.append(angle)
    if len(angle_history) >= 5:
        angle_history.pop(0)
    brake = brake_slider.value * np.mean(np.abs(angle_history))
    
    speed_slider.value = speed_gain_slider.value
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value - brake + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - brake - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

In [8]:
camera.observe(execute, names='value')

In [9]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()